In [1]:
import numpy as np
import strawberryfields as sf
import strawberryfields.ops as ops


# Reproduce 2a, eqn 7

In [2]:
nbar = 1 # mean photon number
v = 2*nbar+1 # v=2, .5 mean photon
#ν := 2n̄ + 1
tmsv_cov = np.array([
    [v,np.sqrt(v**2-1),0,0],
    [np.sqrt(v**2-1),v,0,0],
    [0,0,v,-np.sqrt(v**2-1)],
    [0,0,-np.sqrt(v**2-1),v],
])

In [3]:
# https://strawberryfields.readthedocs.io/en/stable/code/api/strawberryfields.ops.BSgate.html
transmitivity = .52
theta = np.arccos(np.sqrt(transmitivity)) # transmission amplitude
phi = 0#np.pi # phase angle; np.pi/2 is the symmetric beam splitter

In [4]:
# params for state preparations

t_nbar = 10#np.random.uniform(0,12)
d_r = np.random.uniform(-1,1)
d_phi = np.random.uniform(-np.pi,np.pi)
s_r = np.random.uniform(-1,1)
s_phi = np.random.uniform(-1,1)

print(t_nbar,d_r,d_phi)

10 0.9979810822324744 -2.4054485861287995


In [5]:
exp = sf.Program(6) # 1 for the mode goinging through the channel, 2 for the explicit environment

with exp.context as q:
    # q[2] and q[5] is the mode going through our custom thermal loss channel
    
    # Prepare q[2] and q[5] in some state
    ops.Thermal(t_nbar) | q[2]
    ops.Thermal(t_nbar) | q[5]
    ops.Dgate(d_r,d_phi) | q[2]
    ops.Sgate(s_r,s_phi) | q[2]
    
    # Prepare the environment
    ops.Gaussian(tmsv_cov,np.zeros(4)) | [q[0],q[1]] # prepares a gaussian state
    ops.Gaussian(tmsv_cov,np.zeros(4)) | [q[3],q[4]] # prepares a gaussian state

    # Pass 0 and 3 through the channel
    ops.BSgate(theta, phi) | (q[1], q[2]) # applies BS
    ops.BSgate(theta, phi) | (q[4], q[5]) # applies BS
    
    # We will compare our results against the built in Thermal loss channel
    #ops.ThermalLossChannel(transmitivity,nbar) | q[2]
    #ops.ThermalLossChannel(transmitivity,nbar) | q[5]

In [6]:
eng = sf.Engine(backend="gaussian")
#eng = sf.Engine(backend="tf", backend_options={"cutoff_dim": 10})

custom_results = eng.run(exp)

/home/dave/Projects/SF/lib/python3.8/site-packages/strawberryfields/program.py:560: UserWarning: The circuit consists of 2 disconnected components.


In [7]:
exp = sf.Program(2) # we just need one mode since we don't have an explicit environment

with exp.context as q:
    # q[0] is the mode going through the builtin  thermal loss channel
    
    # Prepare q[2] in some state
    ops.Thermal(t_nbar)| q[0]
    ops.Thermal(t_nbar)| q[1]
    #ops.Dgate(d_r,d_phi) | q[0]
    #ops.Sgate(s_r,s_phi) | q[0]

    # We will compare our results against the built in Thermal loss channel
    ops.ThermalLossChannel(transmitivity,nbar) | q[0]
    #ops.ThermalLossChannel(transmitivity,nbar) | q[1]

In [8]:
eng = sf.Engine(backend="gaussian")
#eng = sf.Engine(backend="tf", backend_options={"cutoff_dim": 10})

builtin_results = eng.run(exp)

In [9]:
def build_symplectic(cov,xxpp=True):
    if xxpp==False:
        w = np.array([[0,1],[-1,0]])
        arr = np.zeros_like(cov)
        for ii in range(0,len(cov),2):
            arr[ii:ii+2,ii:ii+2] = w

    if xxpp == True:
        num_modes = len(cov)//2
        arr = np.zeros_like(cov)
        arr[num_modes:,0:num_modes] = -np.eye(num_modes)
        arr[:num_modes,num_modes:] = np.eye(num_modes)
    
    return arr

def g(x,tol=1e-5):
    if x < 1 - tol:
        print('invalid eigenvalue')
        return None
    elif x < 1 + tol:
        return 0
    else:
        return (x+1)/2*np.log2((x+1)/2) - (x-1)/2*np.log2((x-1)/2)

def calculate_entropy(cov):
    omega = build_symplectic(cov)
    #print(omega)
    V = 1j*omega@cov # This is actually the version of V you use to get the eigenvals, not the diagonalized one
    eig = np.linalg.eigvals(V)
    entropy = 0
    for ii in range(len(eig)):
        entropy += g(abs(eig[ii]))
    return entropy/2 # Divide by two because you want the symplectic eigenvalues

In [10]:
def extract_state_cov(cov,state_num):
    num_modes = len(cov)//2
    new_cov = np.zeros((2,2))
    new_cov[0,0] = cov[state_num,state_num]
    new_cov[1,1] = cov[state_num+num_modes,state_num+num_modes]
    new_cov[1,0] = cov[state_num+num_modes,state_num]
    new_cov[0,1] = cov[state_num,state_num+num_modes]
    return new_cov

def extract_state_means(means,state_num):
    num_modes = len(means)//2
    new_means = np.zeros(2)
    new_means[0] = means[state_num]
    new_means[1] = means[state_num+num_modes]
    return new_means

def extract_states_cov(cov,state_list):
    num_modes = len(cov)//2
    subset = state_list + [num_modes+mode for mode in state_list]
    return cov[np.ix_(subset,subset)]

def extract_states_means(means,state_list):
    num_modes = len(means)//2
    subset = state_list + [num_modes+mode for mode in state_list]
    return means[subset]

In [11]:
#custom_results.state.cov()

In [12]:
print(extract_states_cov(custom_results.state.cov(),[2,5]))
print(extract_states_means(custom_results.state.means(),[0,3]))
rho_cov = extract_states_cov(custom_results.state.cov(),[2,5])
env_cov = extract_states_cov(custom_results.state.cov(),[0,1,3,4])
print(calculate_entropy(rho_cov) - calculate_entropy(env_cov))
print(calculate_entropy(rho_cov),calculate_entropy(env_cov))

[[ 2.97092965e+01  0.00000000e+00 -1.35012840e+01  0.00000000e+00]
 [ 0.00000000e+00  1.23600000e+01  0.00000000e+00 -9.93339595e-17]
 [-1.35012840e+01  0.00000000e+00  1.21063804e+01  0.00000000e+00]
 [ 0.00000000e+00 -9.93339595e-17  0.00000000e+00  1.23600000e+01]]
[0. 0. 0. 0.]
-3.351986198346836
8.245441844969612 11.597428043316448


In [13]:
print(builtin_results.state.cov())
print(builtin_results.state.means())
calculate_entropy(builtin_results.state.cov())

[[12.36  0.96  0.    0.  ]
 [ 0.96 21.96  0.    0.  ]
 [ 0.    0.   12.36  0.96]
 [ 0.    0.    0.96 21.96]]
[0. 0. 0. 0.]


8.962799952387353